# 基于MindSpore通过GPT实现情感分类

该实验可进行在线体验，在线体验链接（https://pangu.huaweicloud.com/gallery/asset-detail.html?id=1325b719-fc78-46c4-8f47-9f3623e9b0f4
）

# 环境配置
>
> 此为在线运行平台配置python3.9的指南，如在其他环境平台运行案例，请根据实际情况修改如下代码
>
第一步：设置python版本为3.9.0

In [1]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [2]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

#### 注：以上代码运行完成后，需要重新设置kernel为python-3.9.0

第二步：安装MindSpore框架和MindNLP套件

<div align=center><img src="https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/ai-gallery/change-kernel.PNG"></div>

In [1]:
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.14-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 MB 4.2 MB/s eta 0:00:0000:0100:01



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 68.0 MB/s eta 0:00:0000:0100:01



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 20.8 MB/s eta 0:00:00




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 77.9 MB/s eta 0:00:00ta 0:00:01



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 56.3 MB/s eta 0:00:0000:0100:01










In [2]:
!pip install mindnlp

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 36.5 MB/s eta 0:00:0000:0100:01




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.2 MB/s eta 0:00:00



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.6 MB/s eta 0:00:00



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.1 MB/s eta 0:00:00a 0:00:01



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.6 MB/s eta 0:00:00:00:01



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00a 0:00:01



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.4/773.4 kB 21.3 MB/s eta 0:00:0000:01





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.8 MB/s eta 0:00:00:00:01



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.8 MB/s eta 0:00:0000:0100:01








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 67.3 MB/s eta 0:00:0000:0100:01





     ━━━━━━━━━━━━━━━━━

In [3]:
!pip install jieba

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 75.1 MB/s eta 0:00:0000:0100:01

  Preparing metadata (setup.py) ... done



  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314458 sha256=1a0c4fd06c2092c245959a802f0dea672aea99da620a05636c6461cb6495dd05

  Stored in directory: /home/ma-user/.cache/pip/wheels/2d/22/9e/9af7e8c2773513ac75905acfb75073922bcc1aa176f730a0c9

Successfully built jieba




In [4]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp import load_dataset

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

/home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

  from .autonotebook import tqdm as notebook_tqdm

Building prefix dict from the default dictionary ...

Dumping model to file cache /tmp/jieba.cache

Loading model cost 0.813 seconds.

Prefix dict has been built successfully.


In [5]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']





Generating train split: 100%|██████████| 25000/25000 [00:00<00:00, 126446.58 examples/s]

Generating test split: 100%|██████████| 25000/25000 [00:00<00:00, 140678.02 examples/s]

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 149765.87 examples/s]


In [6]:
imdb_train.get_dataset_size()

25000

In [7]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=4, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'attention_mask': (None, 0)})

    return dataset

In [8]:
from mindnlp.transformers import GPTTokenizer
# tokenizer
gpt_tokenizer = GPTTokenizer.from_pretrained('openai-gpt')

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

100%|██████████| 25.0/25.0 [00:00<00:00, 82.8kB/s]

656B [00:00, 1.61MB/s]                   

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [9]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [10]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [11]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[4, 512], dtype=Int64, value=
 [[  655,   256,   252 ... 40480, 40480, 40480],
  [  249,  3210,   645 ... 40480, 40480, 40480],
  [  616,  4121,   509 ... 40480, 40480, 40480],
  [ 9632,   240,   616 ...   239,   295,  4672]]),
 Tensor(shape=[4, 512], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 1, 1, 1]]),
 Tensor(shape=[4], dtype=Int32, value= [0, 0, 1, 1])]

In [12]:
from mindnlp.transformers import GPTForSequenceClassification
from mindspore.experimental.optim import Adam

# set bert config and define parameters for training
model = GPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

metric = Accuracy()

# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='gpt_imdb_finetune_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_train, metrics=metric,
                  epochs=3, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

100%|██████████| 457M/457M [00:22<00:00, 21.6MB/s] 

100%|██████████| 74.0/74.0 [00:00<00:00, 279kB/s]

The following parameters in models are missing parameter:

['score.weight']


#### 注: 如果想要运行的更快一些，可以在训练时需要V100的算力

In [13]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0: 100%|██████████| 4375/4375 [25:53<00:00,  2.82it/s, loss=0.2551044] 


Checkpoint: 'gpt_imdb_finetune_epoch_0.ckpt' has been saved in epoch: 0.


Evaluate: 100%|██████████| 4375/4375 [11:22<00:00,  6.41it/s]


Evaluate Score: {'Accuracy': 0.9488}

---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 0.---------------


...

Evaluate Score: {'Accuracy': 0.9763428571428572}

---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 1.---------------


Epoch 2: 100%|██████████| 4375/4375 [25:31<00:00,  2.86it/s, loss=0.101588644]


The maximum number of stored checkpoints has been reached.

Checkpoint: 'gpt_imdb_finetune_epoch_2.ckpt' has been saved in epoch: 2.


Evaluate: 100%|██████████| 4375/4375 [10:52<00:00,  6.70it/s]


Evaluate Score: {'Accuracy': 0.9870285714285715}

---------------Best Model: 'gpt_imdb_finetune_best.ckpt' has been saved in epoch: 2.---------------

Loading best model from 'checkpoint' with '['Accuracy']': [0.9870285714285715]...

---------------The model is already load the best model from 'gpt_imdb_finetune_best.ckpt'.---------------


In [14]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate: 100%|██████████| 6250/6250 [15:06<00:00,  6.90it/s]

Evaluate Score: {'Accuracy': 0.93128}
